In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import rlrom
from rlrom.app import *
from rlrom.testers import *
import rlrom.utils as hf

In [2]:
from datetime import timedelta
from pandas import DataFrame

from simglucose.simulation.sim_engine import batch_sim
from simglucose_init import PATIENT_NAMES, build_sim_obj




In [3]:
NUM_SCENARIOS = 1


def batch_simglucose() -> list[DataFrame]:
    perturb = True

    # Define meals as a list of tuples (time, meal_size) where time is the hour in a day in 24-hour format.
    meals = [(7, 45), (12, 70), (16, 15), (18, 80), (23, 10)]
    if perturb:
        # TODO different purterbation strategies
        meals.insert(5, (21, 0))  # add a dummy meal of size 0
        meals.insert(3, (14, 0))  # add a dummy meal of size 0
        meals.insert(1, (10, 0))  # add a dummy meal of size 0

    # Create scenarios
    sim_obj_list = [build_sim_obj(meals, name) for name in PATIENT_NAMES[:NUM_SCENARIOS]]

    # Batch simulation
    return batch_sim(sim_obj_list, parallel=True)




In [4]:
sim_results = batch_simglucose()


Process ID: 64819
Simulation starts ...
Simulation Completed!
Simulation took 2.6348843574523926 sec.


In [5]:
sim_results[0].to_csv('simglucose_results.csv')